In [1]:
import os
os.chdir("../")
%pwd

'd:\\PythonProject\\End-to-EndProjects\\Kidney-Disease-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
from kdClassifier.constants import *
from kdClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_file, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
tf.compat.v1.enable_eager_execution()


[ 2025-01-09 18:11:35,790: WARNING: module_wrapper: From C:\Users\hp\AppData\Local\Temp\ipykernel_16092\2119661200.py:6: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
 ]


In [6]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        print(type(self.train_generator))
        print(type(self.valid_generator))
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[ 2025-01-09 18:11:35,876: INFO: common: yaml file: config\config.yaml loaded successfully ]
[ 2025-01-09 18:11:35,882: INFO: common: yaml file: params.yaml loaded successfully ]
[ 2025-01-09 18:11:35,882: INFO: common: created directory at: artifacts ]
[ 2025-01-09 18:11:35,889: INFO: common: created directory at: artifacts\training ]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
<class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>
<class 'keras.src.legacy.preprocessing.image.DirectoryIterator'>


d:\PythonProject\End-to-EndProjects\Kidney-Disease-Classification\.kdc\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.5083 - loss: 10.6617 - val_accuracy: 0.6125 - val_loss: 2.0090
Epoch 2/5
 1/23 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.8750 - loss: 3.1094

d:\PythonProject\End-to-EndProjects\Kidney-Disease-Classification\.kdc\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 427ms/step - accuracy: 0.8750 - loss: 3.1094 - val_accuracy: 0.4375 - val_loss: 9.5101
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.7453 - loss: 5.1841 - val_accuracy: 0.6000 - val_loss: 8.8067
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 415ms/step - accuracy: 0.6250 - loss: 7.6869 - val_accuracy: 0.4750 - val_loss: 13.7183
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.7247 - loss: 4.3048 - val_accuracy: 0.8750 - val_loss: 0.8474
